In [1]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

# Create a Tool

In [2]:
@tool
def multiply(a: int, b: int) -> int:
    """Given two numbers a and b, return their product."""
    return a * b

In [3]:
multiply.invoke({"a": 2, "b": 3})

6

In [4]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

# Initialize model

In [47]:
from langchain_ollama import ChatOllama

chat = ChatOllama(
    model="llama3.2:latest",
    temperature=.5
)

# Bind the tools

In [48]:
llm_with_tools=chat.bind_tools([multiply])

# Testing if tools are called

In [51]:
llm_with_tools.invoke("Can you multiply 2 and 3?")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-08-09T16:32:15.3852056Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1513029400, 'load_duration': 56980000, 'prompt_eval_count': 175, 'prompt_eval_duration': 66530700, 'eval_count': 22, 'eval_duration': 1388455200, 'model_name': 'llama3.2:latest'}, id='run--038ff91e-9aa0-4cd2-ab74-ddcc070b2564-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': '5367109b-b895-4abd-9028-65b508d24ac8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 175, 'output_tokens': 22, 'total_tokens': 197})

# Check Which tools were called

In [54]:
tool_calls=llm_with_tools.invoke("Can you multiply 2 and 3?").tool_calls
tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': '8f26b3bb-27f8-4ef8-b283-8c8fe97eadb2',
  'type': 'tool_call'}]

# Invoke the tool_message into Custom Tool

In [55]:
multiply.invoke(tool_calls[0])

ToolMessage(content='6', name='multiply', tool_call_id='8f26b3bb-27f8-4ef8-b283-8c8fe97eadb2')

# Combine all and create a full AI response

1. Create A Human message/ Prompt
2. Invoke using LLM and return a AI message
3. Add both message in the list
4. Find the tool were called 
5. Invoke the tool using the tool message extracted from step 4
6. Combine all the messge
7. Send the list of messages to AI
8. Print final result 

In [75]:
message=HumanMessage("What is the multiplication of 2 and 10?")
message=[message]
llm_message=llm_with_tools.invoke(message)

In [69]:
message.append(llm_message)

In [70]:
message

[HumanMessage(content='What is the multiplication of 2 and 10?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-08-09T17:27:02.8126514Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9157557700, 'load_duration': 2257430100, 'prompt_eval_count': 177, 'prompt_eval_duration': 5569765300, 'eval_count': 22, 'eval_duration': 1327185600, 'model_name': 'llama3.2:latest'}, id='run--17f8750d-880e-471c-98b1-91e80ca02b2e-0', tool_calls=[{'name': 'multiply', 'args': {'a': '2', 'b': '10'}, 'id': '241f3d87-f77a-4771-884d-2ecfef4a2e32', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 22, 'total_tokens': 199})]

In [71]:
tool_calls=multiply.invoke(llm_message.tool_calls[0])
tool_calls

ToolMessage(content='20', name='multiply', tool_call_id='241f3d87-f77a-4771-884d-2ecfef4a2e32')

In [72]:
message.append(tool_calls)

In [73]:
message

[HumanMessage(content='What is the multiplication of 2 and 10?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-08-09T17:27:02.8126514Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9157557700, 'load_duration': 2257430100, 'prompt_eval_count': 177, 'prompt_eval_duration': 5569765300, 'eval_count': 22, 'eval_duration': 1327185600, 'model_name': 'llama3.2:latest'}, id='run--17f8750d-880e-471c-98b1-91e80ca02b2e-0', tool_calls=[{'name': 'multiply', 'args': {'a': '2', 'b': '10'}, 'id': '241f3d87-f77a-4771-884d-2ecfef4a2e32', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 22, 'total_tokens': 199}),
 ToolMessage(content='20', name='multiply', tool_call_id='241f3d87-f77a-4771-884d-2ecfef4a2e32')]

In [74]:
llm_with_tools.invoke(message).content

'The result of multiplying 2 and 10 is 20.'